# Writing your own callbacks

## 1. Introduction 

## 2. Setup

In [1]:
import tensorflow as tf
from tensorflow import keras

print("The version of Tensorflow: {}".format(tf.__version__))

The version of Tensorflow: 2.4.1


## 3. Keras callbacks overview

## 4. An overview of callback methods

### i. Global methods

### ii. Batch-level methods for training/testing/predicting

### iii. Epoch-level methods (training only)

## 5. A basic example

In [2]:
# Let's take a look at a concrete example. 
# To get started, let's import tensorflow and define a simple Sequential Keras model:

# Define the Keras model to add callbacks to
def get_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(1, input_dim=784))
    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=0.1),
        loss="mean_squared_error",
        metrics=["mean_absolute_error"],
    )
    return model

In [3]:
# Then, load the MNIST data for training and testing from Keras datasets API:

# Load example MNIST data and pre-process it
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 784).astype("float32") / 255.0
x_test = x_test.reshape(-1, 784).astype("float32") / 255.0

# Limit the data to 1000 samples
x_train = x_train[:1000]
y_train = y_train[:1000]
x_test = x_test[:1000]
y_test = y_test[:1000]

In [4]:
class CustomCallback(keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        keys = list(logs.keys())
        print("Starting training; got log keys: {}".format(keys))

    def on_train_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop training; got log keys: {}".format(keys))

    def on_epoch_begin(self, epoch, logs=None):
        keys = list(logs.keys())
        print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        print("End epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_test_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start testing; got log keys: {}".format(keys))

    def on_test_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop testing; got log keys: {}".format(keys))

    def on_predict_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start predicting; got log keys: {}".format(keys))

    def on_predict_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop predicting; got log keys: {}".format(keys))

    def on_train_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: start of batch {}; got log keys: {}".format(batch, keys))

    def on_train_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: end of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: start of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: end of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: start of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: end of batch {}; got log keys: {}".format(batch, keys))

In [5]:
# Let's try it out:

model = get_model()
model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=1,
    verbose=0,
    validation_split=0.5,
    callbacks=[CustomCallback()],
)

res = model.evaluate(
    x_test, y_test, batch_size=128, verbose=0, callbacks=[CustomCallback()]
)

res = model.predict(x_test, batch_size=128, callbacks=[CustomCallback()])

Starting training; got log keys: []
Start epoch 0 of training; got log keys: []
...Training: start of batch 0; got log keys: []
...Training: end of batch 0; got log keys: ['loss', 'mean_absolute_error']
...Training: start of batch 1; got log keys: []
...Training: end of batch 1; got log keys: ['loss', 'mean_absolute_error']
...Training: start of batch 2; got log keys: []
...Training: end of batch 2; got log keys: ['loss', 'mean_absolute_error']
...Training: start of batch 3; got log keys: []
...Training: end of batch 3; got log keys: ['loss', 'mean_absolute_error']
Start testing; got log keys: []
...Evaluating: start of batch 0; got log keys: []
...Evaluating: end of batch 0; got log keys: ['loss', 'mean_absolute_error']
...Evaluating: start of batch 1; got log keys: []
...Evaluating: end of batch 1; got log keys: ['loss', 'mean_absolute_error']
...Evaluating: start of batch 2; got log keys: []
...Evaluating: end of batch 2; got log keys: ['loss', 'mean_absolute_error']
...Evaluating: 

### Usage of logs dict

In [6]:
class LossAndErrorPrintingCallback(keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_test_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_epoch_end(self, epoch, logs=None):
        print(
            "The average loss for epoch {} is {:7.2f} "
            "and mean absolute error is {:7.2f}.".format(
                epoch, logs["loss"], logs["mean_absolute_error"]
            )
        )


model = get_model()
model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=2,
    verbose=0,
    callbacks=[LossAndErrorPrintingCallback()],
)

res = model.evaluate(
    x_test,
    y_test,
    batch_size=128,
    verbose=0,
    callbacks=[LossAndErrorPrintingCallback()],
)

For batch 0, loss is   31.33.
For batch 1, loss is  442.26.
For batch 2, loss is  303.57.
For batch 3, loss is  229.86.
For batch 4, loss is  185.47.
For batch 5, loss is  155.68.
For batch 6, loss is  134.26.
For batch 7, loss is  120.97.
The average loss for epoch 0 is  120.97 and mean absolute error is    5.99.
For batch 0, loss is    6.83.
For batch 1, loss is    5.83.
For batch 2, loss is    5.48.
For batch 3, loss is    5.15.
For batch 4, loss is    5.13.
For batch 5, loss is    5.06.
For batch 6, loss is    4.90.
For batch 7, loss is    4.95.
The average loss for epoch 1 is    4.95 and mean absolute error is    1.80.
For batch 0, loss is    7.39.
For batch 1, loss is    7.35.
For batch 2, loss is    7.20.
For batch 3, loss is    7.10.
For batch 4, loss is    7.13.
For batch 5, loss is    7.21.
For batch 6, loss is    7.17.
For batch 7, loss is    7.10.


## 6. Usage of self.model attribute

## 7. Examples of Keras callback applications

### i. Early stopping at minimum loss

In [7]:
# tf.keras.callbacks.EarlyStopping provides a more complete and general implementation.

import numpy as np


class EarlyStoppingAtMinLoss(keras.callbacks.Callback):
    """Stop training when the loss is at its min, i.e. the loss stops decreasing.

  Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
  """

    def __init__(self, patience=0):
        super(EarlyStoppingAtMinLoss, self).__init__()
        self.patience = patience
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None

    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get("loss")
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Restoring model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))


model = get_model()
model.fit(
    x_train,
    y_train,
    batch_size=64,
    steps_per_epoch=5,
    epochs=30,
    verbose=0,
    callbacks=[LossAndErrorPrintingCallback(), EarlyStoppingAtMinLoss()],
)

For batch 0, loss is   37.50.
For batch 1, loss is  517.74.
For batch 2, loss is  353.30.
For batch 3, loss is  267.52.
For batch 4, loss is  215.15.
The average loss for epoch 0 is  215.15 and mean absolute error is    8.79.
For batch 0, loss is    6.17.
For batch 1, loss is    5.62.
For batch 2, loss is    5.99.
For batch 3, loss is    5.99.
For batch 4, loss is    5.84.
The average loss for epoch 1 is    5.84 and mean absolute error is    2.00.
For batch 0, loss is    5.41.
For batch 1, loss is    5.47.
For batch 2, loss is    4.82.
For batch 3, loss is    4.90.
For batch 4, loss is    4.85.
The average loss for epoch 2 is    4.85 and mean absolute error is    1.75.
For batch 0, loss is    5.30.
For batch 1, loss is    6.61.
For batch 2, loss is    6.30.
For batch 3, loss is    6.50.
For batch 4, loss is    7.27.
The average loss for epoch 3 is    7.27 and mean absolute error is    2.24.
Restoring model weights from the end of the best epoch.
Epoch 00004: early stopping


### ii. Learning rate scheduling

In [8]:
# See callbacks.LearningRateScheduler for a more general implementations.

class CustomLearningRateScheduler(keras.callbacks.Callback):
    """Learning rate scheduler which sets the learning rate according to schedule.

  Arguments:
      schedule: a function that takes an epoch index
          (integer, indexed from 0) and current learning rate
          as inputs and returns a new learning rate as output (float).
  """

    def __init__(self, schedule):
        super(CustomLearningRateScheduler, self).__init__()
        self.schedule = schedule

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        # Get the current learning rate from model's optimizer.
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        # Call schedule function to get the scheduled learning rate.
        scheduled_lr = self.schedule(epoch, lr)
        # Set the value back to the optimizer before this epoch starts
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print("\nEpoch %05d: Learning rate is %6.4f." % (epoch, scheduled_lr))


LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (3, 0.05),
    (6, 0.01),
    (9, 0.005),
    (12, 0.001),
]


def lr_schedule(epoch, lr):
    """Helper function to retrieve the scheduled learning rate based on epoch."""
    if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
        return lr
    for i in range(len(LR_SCHEDULE)):
        if epoch == LR_SCHEDULE[i][0]:
            return LR_SCHEDULE[i][1]
    return lr


model = get_model()
model.fit(
    x_train,
    y_train,
    batch_size=64,
    steps_per_epoch=5,
    epochs=15,
    verbose=0,
    callbacks=[
        LossAndErrorPrintingCallback(),
        CustomLearningRateScheduler(lr_schedule),
    ],
)


Epoch 00000: Learning rate is 0.1000.
For batch 0, loss is   27.13.
For batch 1, loss is  446.25.
For batch 2, loss is  305.99.
For batch 3, loss is  231.97.
For batch 4, loss is  187.28.
The average loss for epoch 0 is  187.28 and mean absolute error is    8.21.

Epoch 00001: Learning rate is 0.1000.
For batch 0, loss is    7.70.
For batch 1, loss is    7.73.
For batch 2, loss is    7.37.
For batch 3, loss is    6.87.
For batch 4, loss is    6.59.
The average loss for epoch 1 is    6.59 and mean absolute error is    2.11.

Epoch 00002: Learning rate is 0.1000.
For batch 0, loss is    5.35.
For batch 1, loss is    4.90.
For batch 2, loss is    5.17.
For batch 3, loss is    4.60.
For batch 4, loss is    4.35.
The average loss for epoch 2 is    4.35 and mean absolute error is    1.69.

Epoch 00003: Learning rate is 0.0500.
For batch 0, loss is    3.83.
For batch 1, loss is    4.26.
For batch 2, loss is    4.19.
For batch 3, loss is    4.34.
For batch 4, loss is    4.34.
The average loss

### iii. Built-in Keras callbacks

# reference

### https://tensorflow.google.cn/guide/keras/custom_callback